In [1]:
import pandas as pd
import codecs
import re

# 生成训练数据和测试数据

In [2]:
train_df = pd.read_csv('../input/Train_Data.csv')
test_df = pd.read_csv('../input/Test_Data.csv')

def stop_words(x):
    try:
        x = x.strip()
    except:
        return ''
    x = re.sub('\?\?+','',x)
    x = re.sub('\{IMG:.?.?.?\}','',x)
    return x

In [3]:
train_df['text'] =  train_df['title'].fillna('') + train_df['text'].fillna('')
test_df['text'] =  test_df['title'].fillna('') + test_df['text'].fillna('')

train_df['text'] = train_df['text'].apply(stop_words)
test_df['text'] = test_df['text'].apply(stop_words)

train_df = train_df[~train_df['unknownEntities'].isnull()]

In [4]:
with codecs.open('./bert-chinese-ner/data/train.txt', 'w') as up:
    for row in train_df.iloc[:-200].itertuples():
        # print(row.unknownEntities)

        text_lbl = row.text
        entitys = str(row.unknownEntities).split(';')
        for entity in entitys:
            text_lbl = text_lbl.replace(entity, 'Ё' + (len(entity)-1)*'Ж')
        
        for c1, c2 in zip(row.text, text_lbl):
            if c2 == 'Ё':
                up.write('{0} {1}\n'.format(c1, 'B-ORG'))
            elif c2 == 'Ж':
                up.write('{0} {1}\n'.format(c1, 'I-ORG'))
            else:
                up.write('{0} {1}\n'.format(c1, 'O'))
        
        up.write('\n')

In [5]:
with codecs.open('./bert-chinese-ner/data/dev.txt', 'w') as up:
    for row in train_df.iloc[-200:].itertuples():
        # print(row.unknownEntities)

        text_lbl = row.text
        entitys = str(row.unknownEntities).split(';')
        for entity in entitys:
            text_lbl = text_lbl.replace(entity, 'Ё' + (len(entity)-1)*'Ж')
        
        for c1, c2 in zip(row.text, text_lbl):
            if c2 == 'Ё':
                up.write('{0} {1}\n'.format(c1, 'B-ORG'))
            elif c2 == 'Ж':
                up.write('{0} {1}\n'.format(c1, 'I-ORG'))
            else:
                up.write('{0} {1}\n'.format(c1, 'O'))
        
        up.write('\n')

In [6]:
with codecs.open('./bert-chinese-ner/data/test.txt', 'w') as up:
    for row in test_df.iloc[:].itertuples():

        text_lbl = row.text
        for c1 in text_lbl:
            up.write('{0} {1}\n'.format(c1, 'O'))
        
        up.write('\n')

In [ ]:
with codecs.open('./bert-chinese-ner/data/dev.txt', 'w') as up:
    for row in train_df.iloc[-500:].itertuples():
        # print(row.unknownEntities)

        text_lbl = row.text
        entitys = str(row.unknownEntities).split(';')
        for entity in entitys:
            text_lbl = text_lbl.replace(entity, 'Ё' + (len(entity)-1)*'Ж')
        
        for c1, c2 in zip(row.text, text_lbl):
            if c2 == 'Ё':
                up.write('{0} {1}\n'.format(c1, 'B-ORG'))
            elif c2 == 'Ж':
                up.write('{0} {1}\n'.format(c1, 'I-ORG'))
            else:
                up.write('{0} {1}\n'.format(c1, 'O'))
        
        up.write('\n')

In [ ]:
with codecs.open('./bert-chinese-ner/data/test.txt', 'w') as up:
    for row in test_df.iloc[:].itertuples():

        text_lbl = row.text
        for c1 in text_lbl:
            up.write('{0} {1}\n'.format(c1, 'O'))
        
        up.write('\n')

# 生成提交文件

In [9]:
test_pred = codecs.open('./bert-chinese-ner/output/result_dir/label_test.txt').readlines()

pred_tag = []
pred_word = []

pred_line_tag = ''
pred_line_word = ''

for line in test_pred:
    line = line.strip()
    
    if len(line) == 0 or line == '':
        pred_tag.append(pred_line_tag)
        pred_word.append(pred_line_word)
        pred_line_tag = ''
        pred_line_word = ''
        continue
    
    c, _, tag = line.split(' ')
    
    if tag != 'O':
        tag = tag[1:]
        pred_line_word += c
    else:
        pred_line_word += ';'
        
    pred_line_tag += tag    

In [10]:
def filter_word(w):
    for wbad in ['？','《','🔺','️?','!','#','%','%','，','Ⅲ','》','丨','、','）','（','​',
                '👍','。','😎','/','】','-','⚠️','：','✅','㊙️','“',')','(','！','🔥',',']:
        if wbad in w:
            return ''
    return w

with codecs.open('baseline2.csv', 'w') as up:
    up.write('id,unknownEntities\n')
    
    for word, id in zip(pred_word, test_df['id'].values):
        word = set([filter_word(x) for x in word.split(';') if x not in ['', ';'] and len(x) > 1])
        word = [x for x in word if x != '']
        
        if len(word) == 0:
            word = ['我们']
        
        word = ';'.join(list(word))
        up.write('{0},{1}\n'.format(id, word))